In [51]:
import os
from pathlib import Path
import pandas as pd

In [52]:
# paths
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "bengaliai-speech"
INSPECT = INPUT / "inspect"
TRAIN = DATA / "train_mp3s"
TRAIN_WAV = DATA / "train_wavs"
TRAIN_WAV_NOISE_REDUCED = DATA / "train_wavs_noise_reduced"
TEST = DATA / "test_mp3s"
MACRO_NORMALIZATION = INPUT / "macro-normalization"

In [53]:
train = pd.read_csv(DATA / "train.csv")
normalized = pd.read_csv(MACRO_NORMALIZATION / "normalized.csv")
inspect = pd.read_csv(INSPECT / "NISQA_wavfiles.csv")

In [54]:
print(train.head())
print(normalized.head())

             id                                           sentence  split
0  000005f3362c                              ও বলেছে আপনার ঠিকানা!  train
1  00001dddd002                 কোন মহান রাষ্ট্রের নাগরিক হতে চাও?  train
2  00001e0bc131     আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।  train
3  000024b3d810  নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন...  train
4  000028220ab3                                হুমম, ওহ হেই, দেখো।  train
             id                                           sentence  split  \
0  000005f3362c                              ও বলেছে আপনার ঠিকানা!  train   
1  00001dddd002                 কোন মহান রাষ্ট্রের নাগরিক হতে চাও?  train   
2  00001e0bc131     আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।  train   
3  000024b3d810  নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন...  train   
4  000028220ab3                                হুমম, ওহ হেই, দেখো।  train   

                                          normalized  
0                              ও বলেছে

In [55]:
# normalized の id, normalized の列のみ抜き出し
normalized = normalized.loc[:, ["id", "normalized"]]

In [56]:
# train と normalized["normalized"] を id で結合
train = pd.merge(train, normalized, on="id", how="left")

In [57]:
# train の normalized を sentence_normalized に変更
train = train.rename(columns={"normalized": "sentence_normalized"})

In [58]:
# inspect の列 "deg_mp3" を "id" に変更
inspect.rename(columns={"deg_mp3": "id"}, inplace=True)
# inspect の列 "id" の末尾にある .mp3 を削除
inspect["id"] = inspect["id"].str.replace(".mp3", "")
# inspect から 列 "deg" を削除
inspect.drop("deg", axis=1, inplace=True)

In [59]:
inspect.head()

,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,model,id
0,2.810660,3.060386,3.349879,3.030119,3.400320,NISQAv2,67af5635e163
1,1.781602,2.869787,1.852019,2.256710,3.204139,NISQAv2,67ff3e6bf4cf
2,3.304557,3.737251,4.092661,3.244866,3.618690,NISQAv2,6741c3d88006
3,3.193850,3.710290,4.136703,2.662462,3.430386,NISQAv2,67c631a6fbfd
4,2.525036,4.105721,2.742880,2.955755,3.561888,NISQAv2,671616b12c10


In [60]:
# train と inspect を id で結合
train = pd.merge(train, inspect, on="id", how="left")

In [61]:
train.head()

,id,sentence,split,sentence_normalized,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,model
0,000005f3362c,ও বলেছে আপনার ঠিকানা!,train,ও বলেছে আপনার ঠিকানা!,1.186883,1.957373,2.047115,2.520171,3.803196,NISQAv2
1,00001dddd002,কোন মহান রাষ্ট্রের নাগরিক হতে চাও?,train,কোন মহান রাষ্ট্রের নাগরিক হতে চাও?,2.437460,3.635978,2.506423,2.933909,3.477913,NISQAv2
2,00001e0bc131,"আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।",train,"আমি তোমার কষ্টটা বুঝছি, কিন্তু এটা সঠিক পথ না।",3.454123,4.071897,3.892286,3.357083,3.859698,NISQAv2
3,000024b3d810,নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন...,train,নাচ শেষ হওয়ার পর সকলে শরীর ধুয়ে একসঙ্গে ভোজন ক...,4.646200,4.264368,4.742208,4.627210,4.681022,NISQAv2
4,000028220ab3,"হুমম, ওহ হেই, দেখো।",train,"হুমম, ওহ হেই, দেখো।",2.385079,3.696991,2.110702,2.627843,2.049779,NISQAv2


In [ ]:
# train を上書き
train.to_csv(DATA / "train_normalized_with_noise_info.csv", index=False)